Elizabeth Avery Frankenberg; Machine Learning Homework 3; SVM Classification; February 10 2019

In [1]:
#the necessary import statements
import pandas as pd
import numpy as np
import csv
from sklearn.model_selection import KFold, cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, ComplementNB, MultinomialNB, BernoulliNB 
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF

In [2]:
#importing the data and separating it into labels and features
data = pd.read_csv("CS74_HW4_training_set.csv")

data_labels = data['Label']
data_features = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]
data_subset = data[['Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]

#if running multinomial then take the absolute value of all of the data points
#data_features = data_features.abs()
#data_features.head()
data_labels.head()


0    5
1    4
2    2
3    4
4    5
Name: Label, dtype: int64

First, I created a train, test method which runs K fold cross validation on the SVC, NuSVC, MultinomialNB(), ComplementNB() or GaussianNB() classifiers and returns the accuracy obtained.

In [10]:
#this method takes a subset and trains and tests it, returning its score
def train_test(data_subset):
    kf = KFold(n_splits = 10)
    avg_accuracy = []
#  kernel = 1.0 * RBF(1.0)
#     clf = GaussianProcessClassifier(kernel=kernel, random_state=0, multi_class = "one_vs_one")
    clf = svm.SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
        decision_function_shape='ovr', degree=3, gamma=.0000001, kernel='rbf',
        max_iter=-1, probability=False, random_state=None, shrinking=True,
        tol=0.001, verbose=False)
#     clf = clf = svm.NuSVC(nu= 0.5, cache_size=200, class_weight=None, coef0=0.0,
#          decision_function_shape='ovr', degree=3, gamma=.0000001, kernel='rbf',
#          max_iter=-1, probability=False, random_state=None, shrinking=True,
#          tol=0.001, verbose=False)
    #clf = MultinomialNB()
    #clf = GaussianNB()
    #clf = ComplementNB()
    for train, test in kf.split(data_features):
        labels_train, data_train = data_labels.iloc[train], data_subset[train]
        labels_test, data_test = data_labels.iloc[test], data_subset[test]
        #norm_data = preprocessing.normalize(data_train, axis=1)
        #clf = make_pipeline(StandardScaler(), svm.SVC())
        clf.fit(data_train, labels_train) 
        avg_accuracy.append(clf.score(data_test, labels_test))
    avg = 0
    for i in range (0, 10):
        avg += avg_accuracy[i]
    #return avg/10
    return clf

In [22]:
train_test(np.array(data_features))

0.6561111111111111

To try and improve my parameters I created two param selection methods; one that chooses the best parameters for SVC and one that chooses the best for NuSVC. They both conduct GridSearchCV on a specified parameter matrix of Cs, gammas, and alternating kernels for the SVM method and Nu's, gammas and alternating kernels for the NuSVM method. I used this and the subsequent cell to determine the best parameters to use for these two classifiers. I also modified the kernel type, to be linear, sigmoid or polynomial. As these had long run times, I ran them each seperately and then compared the best linear parameters to best sigmoid to best polynomial to best rbf. I found that the rbf kernel provided the best results.

In [5]:
#this method does parameter selection for svc using grid search
def svc_param_selection(features, labels, nfolds):
    Cs = [0.001, .01, .1, 1, 10, 100, 1000]
    gammas = [0.00000001, 0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 1]
    kernels = ['sigmoid'] #can switch this to 'linear', 'sigmoid', or 'polynomial' also, ran these all separately for runtime
    params = {'C': Cs,'gamma' : gammas, 'kernel': kernels}
    grid_search = GridSearchCV(svm.SVC(), params, cv=nfolds)
    grid_search.fit(features, labels)
    grid_search.best_params_
    return grid_search.best_params_

In [6]:
best_params = svc_param_selection(data_features, data_labels, nfolds=10)
print best_params

{'kernel': 'sigmoid', 'C': 1, 'gamma': 1e-06}


The best parameters for SVM were:
kernel = 'rbf', C = 1000, and gamma = 1*e^-7
kernel = 'sigmoid', C = 1, and gamma = 1*e^-6

In [20]:
#this method does parameter selection for nusvc using grid search
def nusvc_param_selection(features, labels, nfolds):
    nus = [.01, .05, .1, .5, .7]
    gammas = [0.00000001, 0.0000001, 0.000001, 0.00001, 0.0001, 0.001, 0.01, 1]
    kernels = ['rbf'] #can switch this to 'linear', 'sigmoid', or 'polynomial' also, ran these all separately for runtime
    params = {'nu': nus,'gamma' : gammas, 'kernel': kernels}
    grid_search = GridSearchCV(svm.NuSVC(), params, cv=nfolds)
    grid_search.fit(features, labels)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
best_params = nusvc_param_selection(data_features, data_labels, nfolds=10)
print best_params

The best parameters for NuSVM were:
kernel = 'rbf', Nu = 0.1, and gamma = 1*e^-6 
kernel = 'sigmoid', Nu = 0.5 and gamma = 1*e^-7


This method then goes through and tries a subset of features. I used this along with some manual attemps to find the best set of subsets possible. 

In [12]:
def try_all_features(data_features):
    best_score= 0
    features = None
    score_dict = dict()
    
    data_features = data_features.abs()
    
    for i in range(0, 6):
        for j in range(0, 6):
            
            if (j < i):
                data_new = np.vstack(np.array(data_features.iloc[:,j:i]))
        
            elif (i == j):
                data_new = np.vstack(np.array(data_features.iloc[:,i]))
        
            else:
                data_new = np.vstack(np.array(data_features.iloc[:,i:j]))
           
            split_avg = train_test(data_new)
            if (split_avg > best_score):
                best_score = split_avg
                features = data_new
                print best_score
    print ("Best score:", best_score)
    print ("Features:", features)

In [13]:
try_all_features(data_features)

0.22222222222222218
0.23433333333333334
0.26888888888888884
0.2696666666666667
0.3137777777777778
('Best score:', 0.3137777777777778)
('Features:', array([[3831.21,   76.  ,  172.  ],
       [1834.2 ,   70.  ,  107.  ],
       [4367.33,  114.  ,  205.  ],
       ...,
       [3454.87,   35.  ,  176.  ],
       [9980.  ,   68.  ,  540.  ],
       [3365.31,   39.  ,  174.  ]]))


Results from different tests:

SVM with all features:
0.65611111
kernel = rbf
C = 1000
gamma = 1*e^-7

0.24488888888888888
kernel = sigmoid
C = 1
gamma = 1*e^-6

NuSVM with all features:
0.6252222
kernel = rbf
Nu = 0.1
gamma = 1*e^-6

0.27822222222222226
kernel = sigmoid
Nu - 0.5
gamma = 1*e^-7

NuSVM with subset of features:
Features = 1-5
0.6094444444444445
kernel = rbf
Nu = 0.1
gamma = 1*e^-6

Features 1-4, 6
0.6201111111111111
Nu = 0.1
gamma = 1*e^-6

Features 1-3, 5-6
0.6091111111111112
Nu = 0.1
gamma = 1*e^-6

Gaussian NB (with trying best feature subset):
Feature 5
0.28366

Multinomial NB (with trying best feature subset and taking absolute value of values):
Features 3-5
0.2703333333333333

Complement NB (with trying best features subset):
Features 3-5
0.3137777777777778

In [4]:
#this method trains the model on a given file
def train(filename):
    data = pd.read_csv(filename)
    data_labels = data['Label']
    data_features = data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]
    return train_test(np.array(data_features))

In [5]:
#this method does the prediction step and puts these values into a new file
def predict_file(C, feature_test_filename, predict_test_filename):
    test_data = pd.read_csv(feature_test_filename)
    with open(predict_test_filename, 'w') as writeFile:
        writer = csv.writer(writeFile) 
        data_features = test_data[['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6']]
        writer.writerow(['Feature_1', 'Feature_2', 'Feature_3', 'Feature_4', 'Feature_5', 'Feature_6', 'Label'])
        X = np.array(data_features)
        line = 0
        while (line < 2161):
            row = X[line]
            prediction = predict(C, row)
            row = np.append(row, prediction)
            writer.writerow(row)
            line += 1

In [6]:
def predict(C, row):
    row = row.reshape(1, -1)
    print row
    prediction = C.predict(row)
    return prediction

These two methods train the model clf using the parameters and type of SVC that I found to be the most accurate. They then predict the labels for the rows in the test file, subsequently outputting them to a predictions.csv. 

In [ ]:
clf = train("CS74_HW4_training_set.csv")
predict_file(clf, "CS74_HW4_test_set.csv", "CS74_HW4_predictions.csv")